In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"

import numpy as np
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')


from ImageD11.parameters import AnalysisSchema
from ImageD11.columnfile import columnfile
import ImageD11.transform, ImageD11.gv_general

In [2]:
pars = AnalysisSchema.from_default().geometry_pars_obj

In [3]:
pars.get_parameters()

{'chi': 0.0,
 'distance': 152736.55305695778,
 'fit_tolerance': 0.05,
 'min_bin_prob': 1e-05,
 'no_bins': 10000,
 'o11': -1,
 'o12': 0,
 'o21': 0,
 'o22': -1,
 'omegasign': 1.0,
 't_x': 0,
 't_y': 0,
 't_z': 0,
 'tilt_x': 0.0,
 'tilt_y': 0.0,
 'tilt_z': 0.0,
 'wavelength': 0.2845704,
 'wedge': 0.0,
 'weight_hist_intensities': 0,
 'y_center': 1049.9295061162281,
 'y_size': 75.0,
 'z_center': 1116.4472483389864,
 'z_size': 75.0}

In [4]:
pars.set('tilt_x', 0.00123)
pars.set('tilt_y', -0.0345)
pars.set('tilt_z', 0.02)
pars.set('chi', 1)
pars.set('wedge', -3)
pars.set('t_x', 1)
pars.set('t_y', 2)
pars.set('t_z', 3)

In [5]:
nrows = 100_000

fc = np.random.random(nrows) * 2048
sc = np.random.random(nrows) * 2048
om = np.random.random(nrows) * 360

In [6]:
cf = columnfile(new=True)
cf.nrows = nrows

cf.addcolumn(fc, 'fc')
cf.addcolumn(sc, 'sc')
cf.addcolumn(om, 'omega')

In [7]:
cf.parameters = pars

In [8]:
cf.updateGeometry()

In [9]:
kxyz_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
cf.addcolumn(kxyz_id11[0], 'kx')
cf.addcolumn(kxyz_id11[1], 'ky')
cf.addcolumn(kxyz_id11[2], 'kz')

In [10]:
cf.titles

['fc',
 'sc',
 'omega',
 'xl',
 'yl',
 'zl',
 'tth',
 'eta',
 'ds',
 'gx',
 'gy',
 'gz',
 'kx',
 'ky',
 'kz']

In [11]:
# in my code, this is:
# (sc, fc): v_det
# omega: omega
# (xl, yl, zl): peak_lab
# (tth, eta): (tth, eta)
# (gx, gy, gz): q_sample
# (kx, ky, kz): q_lab

In [12]:
gvecs = np.column_stack([cf.gx, cf.gy, cf.gz])

In [13]:
import transform as mytrans

In [14]:
import importlib
importlib.reload(mytrans)

<module 'transform' from '/home/esrf/james1997a/Code/Anri/anri/sandbox/transform.py'>

In [15]:
# incident wavevector, normalised
k_in = mytrans._scale_norm_k(np.array([1., 0, 0]), pars.get('wavelength'))
k_in

Array([3.51406893, 0.        , 0.        ], dtype=float64)

In [16]:
origin_sample = jnp.array((pars.get('t_x'), pars.get('t_y'), pars.get('t_z')))
origin_lab = mytrans.sample_to_lab(origin_sample, cf.omega, pars.get('wedge'), pars.get('chi'))

In [17]:
det_trans, beam_cen_shift, x_distance_shift = mytrans.detector_transforms(pars.get('y_center'), pars.get('y_size'), pars.get('tilt_y'),
                                                                          pars.get('z_center'), pars.get('z_size'), pars.get('tilt_z'),
                                                                          pars.get('tilt_x'),
                                                                          pars.get('distance'),
                                                                          pars.get('o11'), pars.get('o12'), pars.get('o21'),pars.get('o22'))

# Lab <-> Detector

In [18]:
v_lab_me = mytrans.det_to_lab(cf.sc, cf.fc, det_trans, beam_cen_shift, x_distance_shift)

In [19]:
assert np.allclose(v_lab_me[:, 0], cf.xl)
assert np.allclose(v_lab_me[:, 1], cf.yl)
assert np.allclose(v_lab_me[:, 2], cf.zl)

In [20]:
v_det_me = mytrans.lab_to_det(cf.xl, cf.yl, cf.zl, det_trans, beam_cen_shift, x_distance_shift)

In [21]:
assert np.allclose(v_det_me[:, 0], cf.sc)
assert np.allclose(v_det_me[:, 1], cf.fc)

# Sample <-> Lab

In [22]:
# map id11 k-vectors (q_lab) into sample frame (q_sample)

In [23]:
q_lab_me = mytrans.q_lab_to_q_sample(kxyz_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [24]:
assert np.allclose(q_lab_me[:, 0], cf.gx)
assert np.allclose(q_lab_me[:, 1], cf.gy)
assert np.allclose(q_lab_me[:, 2], cf.gz)

In [25]:
q_sample_me = mytrans.sample_to_lab(gvecs, cf.omega, pars.get('wedge'), pars.get('chi'))

In [26]:
assert np.allclose(q_sample_me[:, 0], cf.kx)
assert np.allclose(q_sample_me[:, 1], cf.ky)
assert np.allclose(q_sample_me[:, 2], cf.kz)

# k_out

In [27]:
k_out_me_1 = mytrans.q_lab_to_k_out(kxyz_id11.T, k_in)

In [28]:
# should all have length (1/wavelength)
assert jnp.allclose(jnp.linalg.norm(k_out_me_1, axis=1) * pars.get('wavelength'), 1)

In [29]:
k_out_me_2 = mytrans.peak_lab_to_k_out(np.column_stack((cf.xl, cf.yl, cf.zl)), origin_lab, pars.get('wavelength'))

In [30]:
assert np.allclose(k_out_me_1, k_out_me_2)

In [31]:
# unit_vec_lab_me = mytrans.k_out_to_unit_vec_lab(k_out_me_1, origin_lab, pars.get('wavelength'))

In [32]:
# assert jnp.allclose(jnp.linalg.norm(unit_vec_lab_me, axis=1), 1)

In [33]:
sc_me, fc_me = mytrans.q_lab_to_det(kxyz_id11.T, cf.omega, origin_lab, k_in, pars.get('wavelength'), det_trans, beam_cen_shift, x_distance_shift)

In [34]:
sc_me

Array([1955.88976867,  512.46310525, 2003.37264706, ...,  990.80491116,
       2005.35705752,  352.2889853 ], dtype=float64)

In [35]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

# Full pipeline test: sc, fc, omega, origins to g-vectors

In [36]:
q_sample_me = mytrans.det_to_q_sample(cf.sc, cf.fc, cf.omega, origin_lab, k_in,
                                    pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                    det_trans, beam_cen_shift, x_distance_shift)

In [37]:
q_sample_me

Array([[ 0.67009797, -0.36925498, -1.32595603],
       [ 0.02093882,  0.57880578,  0.99079363],
       [ 0.49241396,  0.80027055, -1.37882216],
       ...,
       [-0.21076718, -1.45738774,  0.20346875],
       [ 0.1639296 ,  0.27617701, -1.39299946],
       [ 0.22921437,  1.07346924,  1.16622171]], dtype=float64)

In [38]:
gvecs

array([[ 0.67009797, -0.36925498, -1.32595603],
       [ 0.02093882,  0.57880578,  0.99079363],
       [ 0.49241396,  0.80027055, -1.37882216],
       ...,
       [-0.21076718, -1.45738774,  0.20346875],
       [ 0.1639296 ,  0.27617701, -1.39299946],
       [ 0.22921437,  1.07346924,  1.16622171]])

In [39]:
assert np.allclose(q_sample_me, gvecs)

In [40]:
[sc1_me, sc2_me], [fc1_me, fc2_me], [omega1_me, omega2_me], valid_me = mytrans.q_sample_to_det(gvecs, origin_lab, k_in, 
                                                                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                                                                               det_trans, beam_cen_shift, x_distance_shift)

In [41]:
sc1_me[valid_me]

Array([1979.71322274,  512.46310525, 2033.84784281, ...,  990.80491116,
       2005.35705752,  364.28404176], dtype=float64)

In [42]:
sc2_me[valid_me]

Array([1955.88976867,  505.55477613, 2003.37264706, ...,  962.47316084,
       1997.53321631,  352.2889853 ], dtype=float64)

In [43]:
cf.sc

array([1955.88976867,  512.46310525, 2003.37264706, ...,  990.80491116,
       2005.35705752,  352.2889853 ])

In [44]:
assert np.all(np.isclose(cf.sc, sc1_me[valid_me]) | np.isclose(cf.sc, sc2_me[valid_me]))
assert np.all(np.isclose(cf.fc, fc1_me[valid_me]) | np.isclose(cf.fc, fc2_me[valid_me]))
assert np.all(np.isclose(cf.omega, omega1_me[valid_me]  % 360) | np.isclose(cf.omega, omega2_me[valid_me] % 360))

# Lab <-> tth, eta, omega

In [45]:
tth_id11, eta_id11 = ImageD11.transform.compute_tth_eta_from_xyz(np.stack((cf.xl, cf.yl, cf.zl)), cf.omega, **pars.parameters)

In [46]:
tth_me, eta_me = mytrans.peak_lab_to_tth_eta(jnp.column_stack((cf.xl, cf.yl, cf.zl)),
                                             origin_lab,
                                             k_in,
                                             pars.get('wavelength'))

In [47]:
tth_me, eta_me

(Array([25.16193787, 18.79642995, 27.46742942, ..., 24.42202426,
        23.47203986, 26.34413917], dtype=float64),
 Array([-150.27896992,   28.75789586, -146.03774009, ...,   82.28319751,
         171.30755708,  -39.4328918 ], dtype=float64))

In [48]:
tth_id11, eta_id11

(array([25.16193787, 18.79642995, 27.46742942, ..., 24.42202426,
        23.47203986, 26.34413917]),
 array([-150.27896992,   28.75789586, -146.03774009, ...,   82.28319751,
         171.30755708,  -39.4328918 ]))

In [49]:
assert np.allclose(tth_me, tth_id11)
assert np.allclose(eta_me, eta_id11)

In [50]:
fc_id11, sc_id11 = ImageD11.transform.compute_xyz_from_tth_eta(cf.tth, cf.eta, cf.omega, **pars.parameters)

In [51]:
assert np.allclose(fc_id11, cf.fc)
assert np.allclose(sc_id11, cf.sc)

In [52]:
sc_me, fc_me = mytrans.tth_eta_omega_to_det(cf.tth, cf.eta, cf.omega, origin_lab, k_in,
                                      pars.get('wedge'), pars.get('chi'), pars.get('wavelength'),
                                      det_trans, beam_cen_shift, x_distance_shift)

In [53]:
assert np.allclose(sc_me, cf.sc)
assert np.allclose(fc_me, cf.fc)

In [54]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [55]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [56]:
assert np.allclose(t_id11, t_me.T)

In [57]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T,
                                                                                                      pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 409 ms, sys: 41 ms, total: 450 ms
Wall time: 33.7 ms


In [58]:
%%time

tth_me, (eta1_me, eta2_me), (omega1_me, omega2_me), valid = mytrans.q_sample_to_tth_eta_omega(gvecs, k_in,
                                                                                              pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )

CPU times: user 7.6 s, sys: 720 ms, total: 8.31 s
Wall time: 184 ms


In [59]:
assert np.allclose(omega1_id11, omega1_me)
assert np.allclose(omega2_id11, omega2_me)
assert np.allclose(tth_id11, tth_me)
assert np.allclose(eta1_id11, eta1_me)
assert np.allclose(eta2_id11, eta2_me)


In [60]:
t_id11 = ImageD11.transform.compute_grain_origins(cf.omega, pars.get('wedge'), pars.get('chi'), cf.xl, cf.yl, cf.zl)

In [61]:
t_me = mytrans.sample_to_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), cf.omega, pars.get('wedge'), pars.get('chi'))

In [62]:
t_id11.shape

(3, 100000)

In [63]:
t_me.shape

(100000, 3)

In [64]:
assert np.allclose(t_id11, t_me.T)

In [65]:
%%time

tth_id11, (eta1_id11, eta2_id11), (omega1_id11, omega2_id11) = ImageD11.transform.uncompute_g_vectors(gvecs.T, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

CPU times: user 499 ms, sys: 49.6 ms, total: 549 ms
Wall time: 34.5 ms


In [66]:
tth_id11

array([25.16193787, 18.79642995, 27.46742942, ..., 24.42202426,
       23.47203986, 26.34413917])

In [67]:
eta1_me

Array([152.28171405,  28.75789586, 148.04048423, ...,  82.28319751,
       171.30755708,  41.43563593], dtype=float64)

In [68]:
eta1_id11

array([152.28171405,  28.75789586, 148.04048423, ...,  82.28319751,
       171.30755708,  41.43563593])

In [69]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))

In [70]:
k_id11

array([[-0.33345105, -0.18740762, -0.39612983, ..., -0.31442203,
        -0.2907732 , -0.36495421],
       [ 0.74074368, -0.5447395 ,  0.90547932, ..., -1.43974975,
        -0.21153107,  0.99049617],
       [-1.29755506,  0.99260458, -1.34433753, ...,  0.19509176,
        -1.3835822 ,  1.2044394 ]])

In [71]:
k_me = mytrans.tth_eta_to_q_lab(cf.tth, cf.eta,k_in, pars.get('wavelength'), )

In [72]:
k_me

Array([[-0.33345105,  0.74074368, -1.29755506],
       [-0.18740762, -0.5447395 ,  0.99260458],
       [-0.39612983,  0.90547932, -1.34433753],
       ...,
       [-0.31442203, -1.43974975,  0.19509176],
       [-0.2907732 , -0.21153107, -1.3835822 ],
       [-0.36495421,  0.99049617,  1.2044394 ]], dtype=float64)

In [73]:
k_id11.T

array([[-0.33345105,  0.74074368, -1.29755506],
       [-0.18740762, -0.5447395 ,  0.99260458],
       [-0.39612983,  0.90547932, -1.34433753],
       ...,
       [-0.31442203, -1.43974975,  0.19509176],
       [-0.2907732 , -0.21153107, -1.3835822 ],
       [-0.36495421,  0.99049617,  1.2044394 ]])

In [74]:
k_id11.shape

(3, 100000)

In [75]:
k_me.shape

(100000, 3)

In [76]:
assert np.allclose(k_id11.T, k_me)

In [77]:
# test computation of angles for g-vectors

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))
pre = (C @ W).T

oms_id11 = ImageD11.gv_general.g_to_k(gvecs.T, pars.get('wavelength'), axis=[0,0,1], pre=pre, post=None)

In [78]:
oms_id11

(array([-148.7965535 ,  -26.76637366,  -52.37056519, ...,  159.84659171,
         -84.71123914,  -34.96117673]),
 array([  88.80221216, -168.10759377,  178.33062855, ...,    4.47109334,
        -129.01256351, -175.92735059]),
 array([ True,  True,  True, ...,  True,  True,  True]))

In [79]:
oms_me = mytrans.omega_solns(gvecs, k_in, pars.get('wavelength'), np.array([0,0,1]), pre, jnp.eye(3), )

In [80]:
oms_me

(Array([-148.7965535 ,  -26.76637366,  -52.37056519, ...,  159.84659171,
         -84.71123914,  -34.96117673], dtype=float64),
 Array([  88.80221216, -168.10759377,  178.33062855, ...,    4.47109334,
        -129.01256351, -175.92735059], dtype=float64),
 Array([ True,  True,  True, ...,  True,  True,  True], dtype=bool))

In [81]:
assert np.allclose(oms_id11[0], oms_me[0])
assert np.allclose(oms_id11[1], oms_me[1])
assert np.allclose(oms_id11[2], oms_me[2])

In [82]:
# now use k vectors to test g_from_k

In [83]:
# test k vector computation
k_id11 = ImageD11.transform.compute_k_vectors(cf.tth, cf.eta, pars.get('wavelength'))
g_id11 = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [84]:
g_id11

array([[ 0.67009797,  0.02093882,  0.49241396, ..., -0.21076718,
         0.1639296 ,  0.22921437],
       [-0.36925498,  0.57880578,  0.80027055, ..., -1.45738774,
         0.27617701,  1.07346924],
       [-1.32595603,  0.99079363, -1.37882216, ...,  0.20346875,
        -1.39299946,  1.16622171]])

In [85]:
g_me = mytrans.lab_to_sample(k_id11.T, cf.omega, pars.get('wedge'), pars.get('chi'))

In [86]:
g_me

Array([[ 0.67009797, -0.36925498, -1.32595603],
       [ 0.02093882,  0.57880578,  0.99079363],
       [ 0.49241396,  0.80027055, -1.37882216],
       ...,
       [-0.21076718, -1.45738774,  0.20346875],
       [ 0.1639296 ,  0.27617701, -1.39299946],
       [ 0.22921437,  1.07346924,  1.16622171]], dtype=float64)

In [87]:
assert np.allclose(g_id11.T, g_me)

In [88]:
g_id11_trans = ImageD11.transform.compute_g_from_k(k_id11, cf.omega, pars.get('wedge'), pars.get('chi'))

In [89]:
g_id11_trans

array([[ 0.67009797,  0.02093882,  0.49241396, ..., -0.21076718,
         0.1639296 ,  0.22921437],
       [-0.36925498,  0.57880578,  0.80027055, ..., -1.45738774,
         0.27617701,  1.07346924],
       [-1.32595603,  0.99079363, -1.37882216, ...,  0.20346875,
        -1.39299946,  1.16622171]])

In [90]:
# C @ W -Z works

W =  mytrans.wedgemat(pars.get('wedge'))
C =  mytrans.chimat(pars.get('chi'))

post = C @ W

g_id11_gvgeneral = ImageD11.gv_general.k_to_g(k_id11, cf.omega, axis=np.array([0., 0., -1]), pre=None, post=post)

In [91]:
g_id11_gvgeneral

array([[ 0.67009797,  0.02093882,  0.49241396, ..., -0.21076718,
         0.1639296 ,  0.22921437],
       [-0.36925498,  0.57880578,  0.80027055, ..., -1.45738774,
         0.27617701,  1.07346924],
       [-1.32595603,  0.99079363, -1.37882216, ...,  0.20346875,
        -1.39299946,  1.16622171]])

In [92]:
assert np.allclose(g_id11_trans, g_id11_gvgeneral)

In [93]:
g_id11 = ImageD11.transform.compute_g_vectors(cf.tth, cf.eta, cf.omega, pars.get('wavelength'), pars.get('wedge'), pars.get('chi'))

In [94]:
g_id11

array([[ 0.67009797,  0.02093882,  0.49241396, ..., -0.21076718,
         0.1639296 ,  0.22921437],
       [-0.36925498,  0.57880578,  0.80027055, ..., -1.45738774,
         0.27617701,  1.07346924],
       [-1.32595603,  0.99079363, -1.37882216, ...,  0.20346875,
        -1.39299946,  1.16622171]])

In [95]:
assert np.allclose(gvecs, g_id11.T)

In [96]:
g_me = mytrans.tth_eta_omega_to_q_sample(cf.tth, cf.eta, cf.omega, k_in, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'))

In [97]:
g_me

Array([[ 0.67009797, -0.36925498, -1.32595603],
       [ 0.02093882,  0.57880578,  0.99079363],
       [ 0.49241396,  0.80027055, -1.37882216],
       ...,
       [-0.21076718, -1.45738774,  0.20346875],
       [ 0.1639296 ,  0.27617701, -1.39299946],
       [ 0.22921437,  1.07346924,  1.16622171]], dtype=float64)

In [98]:
assert np.allclose(gvecs, g_me)

In [99]:
assert np.allclose(gvecs, mytrans.sample_to_lab(mytrans.lab_to_sample(gvecs, cf.omega, pars.get('wedge'), pars.get('chi')), cf.omega, pars.get('wedge'), pars.get('chi')))

In [100]:
# g-vectors to (tth, eta, omega)
tth, [eta1, eta_two], [omega1, omega2], valid = mytrans.q_sample_to_tth_eta_omega(gvecs, k_in,
                                                                                     pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )

In [101]:
tth

Array([25.16193787, 18.79642995, 27.46742942, ..., 24.42202426,
       23.47203986, 26.34413917], dtype=float64)

In [102]:
assert np.allclose(tth, cf.tth)
assert all(np.logical_or(np.isclose(cf.eta, eta1), np.isclose(cf.eta, eta_two)))
assert all(np.logical_or(np.isclose(cf.omega, omega1 % 360), np.isclose(cf.omega, omega2 % 360)))

In [103]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_q_sample(tth, eta1, omega1, k_in,
                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [104]:
# (tth, eta, omega) to g-vectors
gvecs_loop = mytrans.tth_eta_omega_to_q_sample(tth, eta_two, omega2, k_in,
                                               pars.get('wedge'), pars.get('chi'), pars.get('wavelength'), )
assert np.allclose(gvecs_loop, gvecs)

In [105]:
k_me = mytrans.peak_lab_to_q_lab(np.column_stack((cf.xl, cf.yl, cf.zl)), origin_lab, k_in, pars.get('wavelength'), )

In [106]:
k_me

Array([[-0.33345105,  0.74074368, -1.29755506],
       [-0.18740762, -0.5447395 ,  0.99260458],
       [-0.39612983,  0.90547932, -1.34433753],
       ...,
       [-0.31442203, -1.43974975,  0.19509176],
       [-0.2907732 , -0.21153107, -1.3835822 ],
       [-0.36495421,  0.99049617,  1.2044394 ]], dtype=float64)

In [107]:
k_out = mytrans.q_lab_to_k_out(k_me, k_in)

In [108]:
origin_lab.shape

(100000, 3)

In [109]:
assert np.allclose(sc_me, cf.sc)

In [110]:
k_me.shape

(100000, 3)

In [111]:
origin_lab.shape

(100000, 3)

In [112]:
xyz_me = mytrans.q_lab_to_peak_lab(np.column_stack((cf.kx, cf.ky, cf.kz)), cf.omega, origin_lab, k_in,
                                   pars.get('wavelength'),
                                   det_trans, beam_cen_shift, x_distance_shift)

In [113]:
assert np.allclose(xyz_me, np.column_stack((cf.xl, cf.yl, cf.zl)))

In [114]:
xyz_me

Array([[154191.86375218,  35909.74131131, -62901.32607042],
       [151669.46324508, -24837.58843281,  45258.43736078],
       [154137.75989929,  44765.27866964, -66455.64007412],
       ...,
       [153794.66226641, -69200.7543497 ,   9380.19548521],
       [155241.4893881 , -10189.71349269, -66634.06045107],
       [149816.22961329,  47124.15442201,  57303.20926934]],      dtype=float64)

In [115]:
cf.xl

array([154191.86375218, 151669.46324508, 154137.75989929, ...,
       153794.66226641, 155241.4893881 , 149816.22961329])